In [1]:
from Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [2]:
dp = DataPreprocessing()

df_preprocessed = dp.preprocess_df(df, rolling_window = "7d", sample_time = "1d")

In [3]:
df_preprocessed.to_parquet("C:/Users/danie/hka_M_preprocessed.parquet")

In [4]:
import pandas as pd
df_preprocessed = pd.read_parquet("C:/Users/danie/hka_M_preprocessed.parquet")

In [3]:
from Feature_Engineering import *
fe = FeatureEngineering(df_preprocessed.drop(columns = ["VOC_diff", "VOC_diff_per_sec", "VOC_CO2_ratio"]), label = "VOC", categorical_features = ["season", "room_number", "dayofweek", "color"])

In [7]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fe.X_train, fe.y_train)

LinearRegression()

In [8]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

pred_test = lr.predict(fe.X_test)

pred_train = lr.predict(fe.X_train)

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_train, y_pred = pred_train)}")

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_test, y_pred = pred_test)}")

Training score: 0.22758044955013662
Training score: 18952915185.845333


In [9]:
fe.X_train.shape

(5680, 84)

In [10]:
time_steps = 7
def create_ds(data, t_steps):
    data = pd.DataFrame(data)
    data_s = data.copy()
    for i in range(time_steps):
        data = pd.concat([data, data_s.shift(-(i+1))], axis = 1)   
    data.dropna(axis=0, inplace=True)
    return data.values

X_train_stepped = create_ds(fe.X_train, 7)

In [11]:
y_train_stepped = create_ds(fe.y_train, 7)

In [31]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional
from keras.optimizers import Adam, SGD

class MyLSTMModel:
    def __init__(self, lstm_units=int(fe.X_train.shape[1]),input_shape=(None, fe.X_train.shape[1])):
        self.model = Sequential()
        self.model.add(LSTM(units=lstm_units, input_shape=input_shape, return_sequences = True))
        self.model.add(Dropout(0.4))
        self.model.add(LSTM(units=lstm_units*2, return_sequences = True))
        self.model.add(Dropout(0.3))
        self.model.add(Bidirectional(LSTM(units=lstm_units)))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(50, activation = "relu"))
        self.model.add(Dense(30, activation = "relu"))
        self.model.add(Dense(15, activation = "relu"))

        self.model.add(Dense(1))

        self.model.compile(optimizer=Adam(learning_rate = 0.05), loss='mse')
    
    def summary(self):
        self.model.summary()

# Instantiate the model
my_model = MyLSTMModel()

# Print the model summary
X_train_array = fe.X_train.values.reshape(fe.X_train.shape[0], 1, fe.X_train.shape[1]).astype(np.float32)
y_train_array = fe.y_train.values.astype(np.float32).reshape(fe.y_train.shape[0], 1, 1)
my_model.model.fit(X_train_array, y_train_array, epochs = 15, batch_size = 35)

Epoch 1/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 306065.3125
Epoch 2/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 128858.4219
Epoch 3/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 118564.8516
Epoch 4/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 101841.4453
Epoch 5/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 73397.0703
Epoch 6/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 63485.3789
Epoch 7/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53853.2773
Epoch 8/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 48120.9453
Epoch 9/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 46740.7148
Epoch 10/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50423.2969
Epoch 11/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43693.6680
Epoch 12/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41935.3672
Epoch 13/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43788.5117
Epoch 14/15
 1/89 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 35341.7695

In [13]:
X_test_array = fe.X_test.values.reshape(fe.X_test.shape[0], 1, fe.X_test.shape[1]).astype(np.float32)
y_test_array = fe.y_test.values.astype(np.float32).reshape(fe.y_test.shape[0], 1, 1)
pred = my_model.model.predict(X_test_array)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [14]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

def evaluate_model(y_true, y_pred):
    eval_df = dict()

    eval_df["r2"] = r2_score(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mape"] = mean_absolute_percentage_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mae"] = mean_absolute_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mse"] = mean_squared_error(y_pred = y_pred, y_true = y_true).round(2)

    return pd.DataFrame([eval_df])

evaluate_model(fe.y_test, pred)


,r2,mape,mae,mse
0,-0.01,0.33,319.36,161415.29


In [15]:
evaluate_model(fe.y_train, my_model.model.predict(X_train_array))

178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step


,r2,mape,mae,mse
0,-0.71,0.7,384.24,207978.17


In [16]:
import plotly.express as px

px.line(x = fe.y_test.index, y = [fe.y_test, pd.DataFrame(pred)[0]])